In [ ]:
from datascience import *
import numpy as np
import matplotlib
import pandas as pd
matplotlib.use('Agg', warn=False)
%matplotlib inline
import matplotlib.pyplot as plots

nba=Table.read_table('20172058-data2.csv')

games=make_array()
for i in np.arange(len(nba.column(0))):
    temp=nba.take(i)
    games=np.append(games,temp.column(4)+temp.column(5)+temp.column(6)+temp.column(7))

nba=nba.with_column('Games',games)

nba=nba.where('Games',are.above(19))

nba


In [ ]:
WinRate=Table().with_columns('HomePct',nba.column('HomePCT')*100,
                             'AwayPct',nba.column('AwayPCT')*100)
WinRate


In [ ]:
WinRate.hist()
plots.xlabel('Win Rate')
plots.title('Home VS Away')

In [ ]:
HomePct_mean=np.sum(WinRate.column('HomePct'))/len(WinRate.column('HomePct'))
AwayPct_mean=np.sum(WinRate.column('AwayPct'))/len(WinRate.column('AwayPct'))

observed_mean_difference=HomePct_mean-AwayPct_mean
observed_mean_difference                    

In [ ]:
Stadium=make_array()
for i in np.arange(len(WinRate.column('HomePct'))):
    Stadium=np.append(Stadium,'Home')

for i in np.arange(len(WinRate.column('AwayPct'))):
    Stadium=np.append(Stadium,'Away')

Stadium     

In [ ]:
Stadium_WinRate=make_array()
Stadium_WinRate=WinRate.column(0)
Stadium_WinRate=np.append(Stadium_WinRate,WinRate.column(1))

Stadium_WinRate           #홈승률 뒤에 어웨이 승률

In [ ]:
Home_Away_WinRate = Table().with_columns(
    'Stadium',Stadium,
    'WinRate',Stadium_WinRate
)
Home_Away_WinRate

In [ ]:
mean_differences = make_array()
for i in np.arange(5000):
  shuffled = Home_Away_WinRate.sample(with_replacement=False).column(1)
  original_shuffled = Home_Away_WinRate.with_column('Shuffled Data',shuffled)
  shuffled_mean_difference = original_shuffled.group('Stadium',np.mean).column(2)
  simulated_diff = shuffled_mean_difference.item(1)-shuffled_mean_difference.item(0)
  mean_differences = np.append(mean_differences,simulated_diff)


In [ ]:
Table().with_column('Difference Between Averages', mean_differences).hist(density=True)
plots.scatter(observed_mean_difference, 0, color='red', s=100)
plots.title('Home Win Rate - Away Win Rate');

In [ ]:
p_value = np.count_nonzero(mean_differences >= observed_mean_difference) / 5000
p_value